# Claycode SIGGRAPH Evaluation

In [14]:
import pandas as pd
import plotly.graph_objects as go

# Read the CSV data
df = pd.read_csv('results.csv')

# Create the 'type' column from the 'filename' column
df['type'] = df['filename'].str.split('-').map(lambda x: "-".join(x[:-1]).replace("_clay", "clay"))

"""
df['type'] = df['type'].replace({'claycode': 'Claycode (R=1)',
                                 'claycode-fragment': 'Claycode (R=2)',
                                 'qr': 'QR',
                                 'qrart': 'Styl. QR',
                                 'claycodeart': 'Styl. Claycode (R=1)',
                                 'barcode': 'Barcode'
                                 })"""
# Filter the relevant columns
df = df[['successful', 'square_dimension_perc', 'type']]

# Group the data by 'square_dimension_perc' and 'type'
grouped = df.groupby(['square_dimension_perc', 'type'])

# Aggregate the 'successful' column
df = grouped['successful'].agg(['sum', 'count']).reset_index()

# Calculate success rate
df['success_rate'] = df['sum'] / df['count']


# Define all pairs to display
pairs_to_display = [
    ['qr2', 'qr'],
    ['qrart2', 'qrart'],
    ['claycode2', 'claycode'],
    ['claycodeart2', 'claycodeart'],
    ['claycode2-fragment', 'claycode-fragment'],
    ['barcode2', 'barcode']
]

# Define color scheme
color_scheme = {
    'claycode': '#1b9e77', 
    'claycodeart': '#1b9e77', 
    'claycode-fragment': '#1b9e77', 
    'qr': '#d95f02', 
    'qrart': '#d95f02', 
    'barcode': '#7570b3',
    'claycode2': '#1b9e77', 
    'claycodeart2': '#1b9e77', 
    'claycode2-fragment': '#1b9e77', 
    'qr2': '#d95f02', 
    'qrart2': '#d95f02', 
    'barcode2': '#7570b3'
}

# Define line dash style (solid for normal, dashed for "art"/"fragment")
line_dash_map = {
    'claycode': 'dot', 
    'claycodeart': 'dot', 
    'claycode-fragment': 'dot', 
    'qr': 'dot', 
    'qrart': 'dot', 
    'barcode': 'dot',
    'claycode2': 'solid', 
    'claycodeart2': 'solid', 
    'claycode2-fragment': 'solid', 
    'qr2': 'solid', 
    'qrart2': 'solid', 
    'barcode2': 'solid'
}

# Define marker symbols for different types
marker_map = {
    'qr': 'square-open',
    'qrart': 'square',
    'claycode': 'hexagram-open',
    'claycodeart': 'hexagram-dot',
    'barcode': 'cross',
    'claycode-fragment': 'star',
    'claycode2': 'hexagram-open',
    'claycodeart2': 'hexagram-dot',
    'claycode2-fragment': 'star',
    'qr2': 'square-open',
    'qrart2': 'square',
    'barcode2': 'cross',
}

# Create the figure
fig = go.Figure()

def hex_to_rgb(hex_color):
    # Remove the hash if it exists
    hex_color = hex_color.lstrip('#')
    # Convert hex to RGB
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return rgb


for pair in pairs_to_display:
    # Get the data for both types in the pair
    trace_1 = df[df['type'] == pair[0]]
    trace_2 = df[df['type'] == pair[1]]
    
    # Add line traces for both types in the pair
    fig.add_trace(go.Scatter(x=trace_1['square_dimension_perc'], y=trace_1['success_rate'], 
                             mode='lines+markers', name=pair[0].replace("2", ""), 
                             line=dict(color=color_scheme.get(pair[0], '#000000'),
                                       dash=line_dash_map.get(pair[0], 'solid')),
                             marker=dict(symbol=marker_map.get(pair[0], 'circle')),
                             showlegend=True))

    # Only add trace_2 to the plot but don't show it in the legend
    fig.add_trace(go.Scatter(x=trace_2['square_dimension_perc'], y=trace_2['success_rate'], 
                             mode='lines+markers', name=pair[0],
                             line=dict(color=f"rgba({','.join(map(str, hex_to_rgb(color_scheme.get(pair[1], '#000000'))))}, 0.2)",
                                       dash=line_dash_map.get(pair[1], 'dot')),
                             marker=dict(symbol=marker_map.get(pair[1], 'circle')),
                             opacity=0.1,
                             showlegend=False))  # Disable showing this in the legend

    # Loop through each point and create areas with green or red color based on the condition
    for i in range(len(trace_1) - 1):
        x_val_1 = trace_1['square_dimension_perc'].iloc[i]
        x_val_2 = trace_1['square_dimension_perc'].iloc[i + 1]
        
        # Align the success rate values between trace_1 and trace_2 for x_val_1 and x_val_2
        y_val_1_1 = trace_1['success_rate'].iloc[i]
        y_val_2_1 = trace_2[trace_2['square_dimension_perc'] == x_val_1]['success_rate'].iloc[0]
        
        y_val_1_2 = trace_1['success_rate'].iloc[i + 1]
        y_val_2_2 = trace_2[trace_2['square_dimension_perc'] == x_val_2]['success_rate'].iloc[0]
        
        # Ensure both traces' y-values are taken for each x-value, then calculate the fill color
        fill_color = 'rgba(0, 255, 0, 0.1)' if y_val_2_1 <= y_val_1_1 and y_val_2_2 <= y_val_1_2 else 'rgba(255, 0, 0, 0.1)'
        
        # Add the shaded area for this segment (the area between the two traces)
        fig.add_trace(go.Scatter(
            x=[x_val_1, x_val_2, x_val_2, x_val_1], 
            y=[y_val_1_1, y_val_1_2, y_val_2_2, y_val_2_1],
            fill='toself', fillcolor=fill_color, mode='none',
            showlegend=False, line=dict(color='rgba(0, 0, 0, 0)')
        ))

fig.update_layout(
    xaxis_title="Square side dimension (%)",
    yaxis_title="Success rate (%)",
    height=400,
    width=700,
    margin=dict(t=30, b=50, l=50, r=50),  # Adjust margins
    legend=dict(
        x=0.7,  # Set x position inside the plot (adjust as needed)
        y=0.9,  # Set y position inside the plot (adjust as needed)
        xanchor='left',  # Anchor to the left of the legend box
        yanchor='top',   # Anchor to the top of the legend box
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='black',  # Optional: border color for the legend box
        borderwidth=1  # Corrected: Set border width for the legend box
    )
)

# Make lines semi-transparent and thicker
fig.update_traces(opacity=0.7, line=dict(width=2.3))

# Bring markers to the foreground (higher zorder than the lines)
fig.update_traces(marker=dict(size=10, opacity=1), selector=dict(mode='lines+markers'))

# Update x and y axes to go from 0% to 100%, slightly adjusted to avoid cutting off
fig.update_xaxes(range=[0.09,0.52], tickformat='.0%')
fig.update_yaxes(range=[-0.05, 1.05], tickformat='.0%')

fig.write_image("square-and-sinusoidal.pdf", format="pdf", engine="kaleido")
fig.show()